In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
input_size = 28
num_classes = 10
num_epochs = 3
batch_size = 64

train_dataset = datasets.MNIST(
    root='./dataset',
    train=True, 
    transform=transforms.ToTensor(),
    download=True,
)

test_dataset = datasets.MNIST(
    root="./dataset",
    train=False,
    transform=transforms.ToTensor(),
    download=True,
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:06<00:00, 1635848.95it/s]


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 571494.52it/s]


Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 5383508.11it/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2377749.47it/s]

Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



In [3]:
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True
)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        # 全连接层得到结果
        self.out = nn.Linear(32*7*7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [8]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()

    return rights, len(labels)

In [10]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(num_epochs):
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):
        net.train()

        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            # calculate the accuracy
            train_r = (
                sum([tup[0] for tup in train_rights]),
                sum([tup[1] for tup in train_rights]),
            )
            val_r = (
                sum([tup[0] for tup in val_rights]),
                sum([tup[1] for tup in val_rights]),
            )

            print(
                "epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTrain Acc: {:.2f}\tVal Acc: {:.2f}".format(
                    epoch,
                    batch_idx * batch_size,
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                    100.0 * train_r[0].numpy() / train_r[1],
                    100.0 * val_r[0].numpy() / val_r[1],
                )
            )

epoch: 0 [0/60000 (0%)]	Loss: 2.301990	Train Acc: 10.94	Val Acc: 11.80
epoch: 0 [6400/60000 (11%)]	Loss: 0.290464	Train Acc: 79.12	Val Acc: 91.93
epoch: 0 [12800/60000 (21%)]	Loss: 0.195998	Train Acc: 85.62	Val Acc: 94.65
epoch: 0 [19200/60000 (32%)]	Loss: 0.107638	Train Acc: 88.74	Val Acc: 96.00
epoch: 0 [25600/60000 (43%)]	Loss: 0.148959	Train Acc: 90.53	Val Acc: 96.63
epoch: 0 [32000/60000 (53%)]	Loss: 0.209575	Train Acc: 91.81	Val Acc: 97.10
epoch: 0 [38400/60000 (64%)]	Loss: 0.055682	Train Acc: 92.68	Val Acc: 97.07
epoch: 0 [44800/60000 (75%)]	Loss: 0.124660	Train Acc: 93.36	Val Acc: 97.46
epoch: 0 [51200/60000 (85%)]	Loss: 0.077813	Train Acc: 93.85	Val Acc: 97.77
epoch: 0 [57600/60000 (96%)]	Loss: 0.057395	Train Acc: 94.29	Val Acc: 97.62
epoch: 1 [0/60000 (0%)]	Loss: 0.017778	Train Acc: 100.00	Val Acc: 98.21
epoch: 1 [6400/60000 (11%)]	Loss: 0.073651	Train Acc: 97.96	Val Acc: 97.69
epoch: 1 [12800/60000 (21%)]	Loss: 0.078416	Train Acc: 97.92	Val Acc: 98.14
epoch: 1 [19200/60000 (

KeyboardInterrupt: 